# This notebook shows the preprocessing steps

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [52]:
df=pd.read_csv('train.csv',delimiter=',')

In [53]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Data Preprocessing

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [55]:
def show_missing(df):
    missing = df.columns[df.isnull().any()].tolist()
    return missing

In [56]:
missing=show_missing(df)
print(df[missing].isnull().sum())

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


### Columns with more missing values
<p> Since there are 1460 columns in the dataset and the columns namely Alley, PoolQC, Fence and MiscFeatures have more than 80% missing values it is almost impossible to replace them with appropriate values. So I am replacing them with None .</p>

In [57]:
df.loc[df['Alley'].isnull(),'Alley'] = 0.0
df.loc[df['PoolQC'].isnull(),'PoolQC'] = 0.0
df.loc[df['Fence'].isnull(),'Fence'] = 0.0
df.loc[df['MiscFeature'].isnull(),'MiscFeature'] = 0.0

### Replacing null values

#### 1. LotFrontage

In [58]:
print("Correlation of LotFrontage with MSSubClass            =",df['LotFrontage'].corr(df['MSSubClass']))
print("Correlation of LotFrontage with LotArea               =",df['LotFrontage'].corr(df['LotArea']))
print("Correlation of LotFrontage with square root of LotArea=",df['LotFrontage'].corr(np.sqrt(df['LotArea'])))

Correlation of LotFrontage with MSSubClass            = -0.38634688534492884
Correlation of LotFrontage with LotArea               = 0.42609501877180833
Correlation of LotFrontage with square root of LotArea= 0.6020022167939361


In [59]:
condition = df['LotFrontage'].isnull()
df.LotFrontage[condition]=np.sqrt(df.LotArea[condition])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### 2. MasVnrType and MasVnrArea
<p>There are only 8 missing values, so I am replacing them with most frequent value.</p>

In [60]:
df['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [61]:
df.loc[df['MasVnrType'].isnull(),'MasVnrType'] = 0

In [62]:
df['MasVnrArea'].value_counts().idxmax()

0.0

In [63]:
df.loc[df['MasVnrArea'].isnull(),'MasVnrArea'] = 0.0

### Basement
<p> Where the value is null, values in BsmtFinSF1, BsmtFinSF2 is 0. This means there is no basement.</p>

In [64]:
basement_cols=['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2']
df[basement_cols][df['BsmtQual'].isnull()==True]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFinSF1,BsmtFinSF2
17,NaN,NaN,NaN,NaN,NaN,0,0
39,NaN,NaN,NaN,NaN,NaN,0,0
90,NaN,NaN,NaN,NaN,NaN,0,0
102,NaN,NaN,NaN,NaN,NaN,0,0
156,NaN,NaN,NaN,NaN,NaN,0,0
182,NaN,NaN,NaN,NaN,NaN,0,0
259,NaN,NaN,NaN,NaN,NaN,0,0
342,NaN,NaN,NaN,NaN,NaN,0,0
362,NaN,NaN,NaN,NaN,NaN,0,0
371,NaN,NaN,NaN,NaN,NaN,0,0


In [65]:
df.loc[df['BsmtQual'].isnull(),'BsmtQual'] = 0
df.loc[df['BsmtCond'].isnull(),'BsmtCond'] = 0
df.loc[df['BsmtExposure'].isnull(),'BsmtExposure'] = 0
df.loc[df['BsmtFinType1'].isnull(),'BsmtFinType1'] = 0
df.loc[df['BsmtFinType2'].isnull(),'BsmtFinType2'] = 0

### Electrical
<p>Again only 1 missing value, so replacing it with most frequent value</p>

In [66]:
df['Electrical'].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [67]:
df.loc[df['Electrical'].isnull(),'Electrical'] = 'SBrkr'

### FireplaceQu

In [68]:
df.loc[df['FireplaceQu'].isnull(),'FireplaceQu'] = 0

### Garage
<p>Same as basement</p>

In [69]:
garage_cols=['GarageType','GarageQual','GarageCond','GarageYrBlt','GarageFinish','GarageCars','GarageArea']
df[garage_cols][df['GarageType'].isnull()==True]

,GarageType,GarageQual,GarageCond,GarageYrBlt,GarageFinish,GarageCars,GarageArea
39,NaN,NaN,NaN,NaN,NaN,0,0
48,NaN,NaN,NaN,NaN,NaN,0,0
78,NaN,NaN,NaN,NaN,NaN,0,0
88,NaN,NaN,NaN,NaN,NaN,0,0
89,NaN,NaN,NaN,NaN,NaN,0,0
99,NaN,NaN,NaN,NaN,NaN,0,0
108,NaN,NaN,NaN,NaN,NaN,0,0
125,NaN,NaN,NaN,NaN,NaN,0,0
127,NaN,NaN,NaN,NaN,NaN,0,0
140,NaN,NaN,NaN,NaN,NaN,0,0


In [70]:
df.loc[df['GarageType'].isnull(),'GarageType'] = 0
df.loc[df['GarageQual'].isnull(),'GarageQual'] = 0
df.loc[df['GarageCond'].isnull(),'GarageCond'] = 0
df.loc[df['GarageYrBlt'].isnull(),'GarageYrBlt'] = 0
df.loc[df['GarageFinish'].isnull(),'GarageFinish'] = 0

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non

In [72]:
le = LabelEncoder()
for col in df.columns:
    if(df[col].dtype == object):
        le.fit(np.array(df[col].unique(),dtype=np.str))
        df[col]=le.transform(np.array(df[col],dtype=np.str))    

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null int64
Alley            1460 non-null int64
LotShape         1460 non-null int64
LandContour      1460 non-null int64
Utilities        1460 non-null int64
LotConfig        1460 non-null int64
LandSlope        1460 non-null int64
Neighborhood     1460 non-null int64
Condition1       1460 non-null int64
Condition2       1460 non-null int64
BldgType         1460 non-null int64
HouseStyle       1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null int64
RoofMatl         1460 non-null int64
Exterior1st      1460 non-null int64
Ext

In [74]:
df.to_csv('ProcessedTrain.csv')